In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from scipy import stats
import re
import numpy as np

In [2]:
# Set display options to remove limits
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
# Load and clean the datasets
mcdonalds_df = pd.read_csv('mcdonalds_menu.csv')
starbucks_df = pd.read_csv('starbucks_drinkMenu_expanded.csv')

mcdonalds_df.columns = mcdonalds_df.columns.str.strip()
starbucks_df.columns = starbucks_df.columns.str.strip()

In [4]:

# Step 1: Load the dataset
mcdonalds_df = pd.read_csv('mcdonalds_menu.csv')

# Step 2: Filter to get only drinks categories
drink_categories = ['Coffee & Tea', 'Beverages', 'Smoothies & Shakes']
mcdonalds_drinks = mcdonalds_df[mcdonalds_df['Category'].isin(drink_categories)]

# Step 3: Further filter the drinks to get specific serving sizes
drink_sizes = ['12 fl oz', '16 fl oz','20 fl oz', '22 fl oz']
filtered_drinks = mcdonalds_drinks[mcdonalds_drinks['Serving Size'].str.contains('|'.join(drink_sizes), case=False, na=False)]

# Step 4: Filter to keep only the specified items using partial matches
specific_items_keywords = [
    'Coffee', 'Nonfat Latte', 'Mocha with Nonfat Milk', 'Frappé Caramel',
    'Hot Chocolate with Nonfat Milk', 'Regular Iced Coffee', 'Frappé Chocolate Chip',
    'Frappé Mocha', 'Sweet Tea', 'Strawberry Banana Smoothie', 'Nonfat French Vanilla Latte'
]

# Create a regular expression pattern that matches any of the keywords
pattern = '|'.join(specific_items_keywords)

# Filter the DataFrame to keep rows where the 'Item' column contains any of the keywords
final_filtered_drinks = filtered_drinks[filtered_drinks['Item'].str.contains(pattern, case=False, na=False)]

# Step 5: Remove the bracketed parts from the item names
final_filtered_drinks['Item'] = final_filtered_drinks['Item'].str.replace(r'\s*\([^)]*\)', '', regex=True)

# Step 6: Remove the specific items
items_to_remove = [
    'Caramel Iced Coffee', 'French Vanilla Iced Coffee', 'Hazelnut Iced Coffee',
    'Iced Coffee with Sugar Free French Vanilla Syrup', 'Iced Mocha with Nonfat Milk',
    'Nonfat Latte with Sugar Free French Vanilla Syrup'
]
final_filtered_drinks = final_filtered_drinks[~final_filtered_drinks['Item'].isin(items_to_remove)]

# Step 7: Reset the index and order by item name
final_filtered_drinks = final_filtered_drinks.sort_values(['Item', 'Serving Size']).reset_index(drop=True)

# Display the final filtered drinks
drinks_final_md = final_filtered_drinks
drinks_final_md

C:\Users\musak\AppData\Local\Temp\ipykernel_113372\2681074452.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_filtered_drinks['Item'] = final_filtered_drinks['Item'].str.replace(r'\s*\([^)]*\)', '', regex=True)


,Category,Item,Serving Size,Calories,Calories from Fat,Total Fat,Total Fat (% Daily Value),Saturated Fat,Saturated Fat (% Daily Value),Trans Fat,Cholesterol,Cholesterol (% Daily Value),Sodium,Sodium (% Daily Value),Carbohydrates,Carbohydrates (% Daily Value),Dietary Fiber,Dietary Fiber (% Daily Value),Sugars,Protein,Vitamin A (% Daily Value),Vitamin C (% Daily Value),Calcium (% Daily Value),Iron (% Daily Value)
0,Coffee & Tea,Coffee,12 fl oz cup,0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Coffee & Tea,Coffee,16 fl oz cup,0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Coffee & Tea,Coffee,20 fl oz cup,0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Coffee & Tea,Frappé Caramel,12 fl oz cup,450,170,19.0,29,12.0,60,1.0,65,22,125,5,64,21,0,0,57,7,15,0,25,2
4,Coffee & Tea,Frappé Caramel,16 fl oz cup,550,200,23.0,35,15.0,73,1.0,80,27,160,7,79,26,0,0,71,9,20,0,30,2
5,Coffee & Tea,Frappé Caramel,22 fl oz cup,670,250,27.0,42,17.0,87,1.5,95,32,190,8,96,32,0,0,88,11,20,0,35,2
6,Coffee & Tea,Frappé Chocolate Chip,12 fl oz cup,530,200,23.0,35,14.0,72,1.0,65,22,135,6,76,25,1,5,67,8,15,0,25,4
7,Coffee & Tea,Frappé Chocolate Chip,16 fl oz cup,630,240,26.0,41,17.0,85,1.0,80,26,160,7,91,30,1,5,81,9,15,0,30,4
8,Coffee & Tea,Frappé Chocolate Chip,22 fl oz cup,760,280,31.0,48,20.0,101,1.5,95,32,200,8,111,37,1,5,99,12,20,0,35,6
9,Coffee & Tea,Frappé Mocha,12 fl oz cup,450,160,18.0,28,12.0,59,1.0,65,21,125,5,65,22,1,3,57,7,15,0,20,4


In [5]:
drinks_final_md.columns

Index(['Category', 'Item', 'Serving Size', 'Calories', 'Calories from Fat',
       'Total Fat', 'Total Fat (% Daily Value)', 'Saturated Fat',
       'Saturated Fat (% Daily Value)', 'Trans Fat', 'Cholesterol',
       'Cholesterol (% Daily Value)', 'Sodium', 'Sodium (% Daily Value)',
       'Carbohydrates', 'Carbohydrates (% Daily Value)', 'Dietary Fiber',
       'Dietary Fiber (% Daily Value)', 'Sugars', 'Protein',
       'Vitamin A (% Daily Value)', 'Vitamin C (% Daily Value)',
       'Calcium (% Daily Value)', 'Iron (% Daily Value)'],
      dtype='object')

In [6]:

# Step 1: Load the dataset
starbucks_drinks = pd.read_csv('starbucks_drinkMenu_expanded.csv')

# Step 2: Define a mapping from size to serving size
size_mapping = {
    'Tall': '12 fl oz cup',
    'Grande': '16 fl oz cup',
    'Venti': '20 fl oz cup'
}

# Step 3: Create a function to map the size descriptions to serving sizes
def map_size_to_serving(size_description):
    for size, serving in size_mapping.items():
        if size in size_description:
            return serving
    return size_description  # Return original if no match

# Step 4: Apply the mapping function to the 'Beverage_prep' column
starbucks_drinks['Beverage_prep'] = starbucks_drinks['Beverage_prep'].apply(map_size_to_serving)

# Step 5: Filter the DataFrame to include only the desired sizes
desired_sizes = ['12 fl oz cup', '16 fl oz cup', '20 fl oz cup']
size_pattern = '|'.join(desired_sizes)
sb_all = starbucks_drinks[starbucks_drinks['Beverage_prep'].str.contains(size_pattern, case=False, na=False)]

# Step 6: Add priority for nonfat versions
sb_all['Priority'] = sb_all['Beverage_prep'].apply(lambda x: 1 if 'Nonfat' in x else 0)

# Step 7: Modify the 'Beverage' name if 'Beverage_category' contains 'Frappuccino'
def add_frappuccino_prefix(row):
    if 'Frappuccino' in row['Beverage_category']:
        row['Beverage'] = f"Frappuccino {row['Beverage']}"
    return row

sb_all = sb_all.apply(add_frappuccino_prefix, axis=1)

# Step 8: Sort and remove duplicates
sb_filtered = sb_all.sort_values(by=['Beverage', 'Priority'], ascending=[True, False]).drop_duplicates(subset=['Beverage', 'Beverage_prep'], keep='first').drop(columns='Priority')

# Step 9: Reset index and sort by Beverage name
sb_filtered = sb_filtered.reset_index(drop=True).sort_values(['Beverage', 'Beverage_prep'])

# Step 10: Define the exact list of items to keep
specific_items = [
    'Brewed Coffee',
    'Caffè Latte',
    'Caffè Mocha (Without Whipped Cream)',
    'Frappuccino Caramel (Without Whipped Cream)',
    'Hot Chocolate (Without Whipped Cream)',
    'Iced Brewed Coffee (With Milk & Classic Syrup)',
    'Frappuccino Java Chip (Without Whipped Cream)',
    'Frappuccino Mocha (Without Whipped Cream)',
    'Shaken Iced Tazo® Tea Lemonade (With Classic Syrup)',
    'Strawberry Banana Smoothie',
    'Vanilla Latte (Or Other Flavoured Latte)'
]

# Step 11: Filter the DataFrame to keep rows where the 'Beverage' column contains any of the keywords
final_filtered_drinks = sb_filtered[sb_filtered['Beverage'].apply(lambda x: any(item in x for item in specific_items))]

# Display the final sorted drinks

# Step 10: Sort the DataFrame by 'Beverage' and 'Beverage_prep'
drinks_final_sb = final_filtered_drinks.sort_values(['Beverage', 'Beverage_prep']).reset_index(drop=True)
drinks_final_sb

C:\Users\musak\AppData\Local\Temp\ipykernel_113372\1641588518.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb_all['Priority'] = sb_all['Beverage_prep'].apply(lambda x: 1 if 'Nonfat' in x else 0)


,Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
0,Coffee,Brewed Coffee,12 fl oz cup,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
1,Coffee,Brewed Coffee,16 fl oz cup,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
2,Coffee,Brewed Coffee,20 fl oz cup,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
3,Classic Espresso Drinks,Caffè Latte,12 fl oz cup,100,0.2,0.2,0.0,5,120,15,0,14,10.0,15%,0%,30%,0%,75
4,Classic Espresso Drinks,Caffè Latte,16 fl oz cup,130,0.3,0.2,0.0,5,150,19,0,18,13.0,20%,0%,40%,0%,150
5,Classic Espresso Drinks,Caffè Latte,20 fl oz cup,170,0.4,0.3,0.0,10,190,25,0,23,16.0,30%,0%,50%,0%,150
6,Classic Espresso Drinks,Caffè Mocha (Without Whipped Cream),12 fl oz cup,170,2,1.0,0.0,5,100,32,1,27,10.0,15%,0%,25%,20%,95
7,Classic Espresso Drinks,Caffè Mocha (Without Whipped Cream),16 fl oz cup,220,2.5,1.5,0.0,5,125,43,2,34,13.0,20%,0%,35%,25%,175
8,Classic Espresso Drinks,Caffè Mocha (Without Whipped Cream),20 fl oz cup,280,3,2.0,0.0,10,160,54,2,44,17.0,25%,0%,45%,30%,180
9,Frappuccino® Blended Coffee,Frappuccino Caramel (Without Whipped Cream),12 fl oz cup,180,0.1,0.0,0.0,0,160,42,0,41,3.0,4%,0%,10%,0%,70


In [7]:
drinks_final_sb.columns

Index(['Beverage_category', 'Beverage', 'Beverage_prep', 'Calories',
       ' Total Fat (g)', 'Trans Fat (g) ', 'Saturated Fat (g)', ' Sodium (mg)',
       ' Total Carbohydrates (g) ', 'Cholesterol (mg)', ' Dietary Fibre (g)',
       ' Sugars (g)', ' Protein (g) ', 'Vitamin A (% DV) ', 'Vitamin C (% DV)',
       ' Calcium (% DV) ', 'Iron (% DV) ', 'Caffeine (mg)'],
      dtype='object')

In [8]:
drinks_final_md = drinks_final_md.rename(columns={
    'Item': 'Beverage',
    'Serving Size': 'Size',
    'Total Fat': 'Total Fat (g)',
    'Saturated Fat': 'Saturated Fat (g)',
    'Trans Fat': 'Trans Fat (g)',
    'Cholesterol': 'Cholesterol (mg)',
    'Sodium': 'Sodium (mg)',
    'Carbohydrates': 'Total Carbohydrates (g)',
    'Dietary Fiber': 'Dietary Fibre (g)',
    'Sugars': 'Sugars (g)',
    'Protein': 'Protein (g)',
    'Vitamin A (% Daily Value)': 'Vitamin A (% DV)',
    'Vitamin C (% Daily Value)': 'Vitamin C (% DV)',
    'Calcium (% Daily Value)': 'Calcium (% DV)',
    'Iron (% Daily Value)': 'Iron (% DV)'
})

drinks_final_sb = drinks_final_sb.rename(columns={
    'Beverage_prep': 'Size',
    'Beverage_category': 'Category',
    ' Total Fat (g)': 'Total Fat (g)',
    'Trans Fat (g) ': 'Trans Fat (g)',
    ' Sodium (mg)': 'Sodium (mg)',
    ' Total Carbohydrates (g) ': 'Total Carbohydrates (g)',
    ' Dietary Fibre (g)': 'Dietary Fibre (g)',
    ' Sugars (g)': 'Sugars (g)',
    ' Protein (g) ': 'Protein (g)',
    'Vitamin A (% DV) ': 'Vitamin A (% DV)',
    'Vitamin C (% DV)': 'Vitamin C (% DV)',
    ' Calcium (% DV) ': 'Calcium (% DV)',
    'Iron (% DV) ': 'Iron (% DV)'
})

# Remove leading/trailing whitespaces from column names
drinks_final_md.columns = drinks_final_md.columns.str.strip()
drinks_final_sb.columns = drinks_final_sb.columns.str.strip()

In [9]:
drinks_final_md['Restaurant'] = 'McDonalds'
drinks_final_sb['Restaurant'] = 'Starbucks'

In [10]:
common_columns = ['Restaurant', 'Category', 'Beverage', 'Size', 'Calories', 'Total Fat (g)', 
                  'Trans Fat (g)', 'Saturated Fat (g)', 'Sodium (mg)', 'Total Carbohydrates (g)', 
                  'Cholesterol (mg)', 'Dietary Fibre (g)', 'Sugars (g)', 'Protein (g)', 
                  'Vitamin A (% DV)', 'Vitamin C (% DV)', 'Calcium (% DV)', 'Iron (% DV)']

# Select only the common columns from both datasets
drinks_final_md_common = drinks_final_md[common_columns]
drinks_final_sb_common = drinks_final_sb[common_columns]

# Combine the datasets

combined_drinks = pd.concat([drinks_final_md_common, drinks_final_sb_common], ignore_index=True)
combined_drinks = combined_drinks[~combined_drinks['Beverage'].str.contains('Iced Brewed Coffee \(With Classic Syrup\)', case=False, na=False)]

combined_drinks

,Restaurant,Category,Beverage,Size,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV)
0,McDonalds,Coffee & Tea,Coffee,12 fl oz cup,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0
1,McDonalds,Coffee & Tea,Coffee,16 fl oz cup,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0
2,McDonalds,Coffee & Tea,Coffee,20 fl oz cup,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0
3,McDonalds,Coffee & Tea,Frappé Caramel,12 fl oz cup,450,19.0,1.0,12.0,125,64,65,0,57,7.0,15,0,25,2
4,McDonalds,Coffee & Tea,Frappé Caramel,16 fl oz cup,550,23.0,1.0,15.0,160,79,80,0,71,9.0,20,0,30,2
5,McDonalds,Coffee & Tea,Frappé Caramel,22 fl oz cup,670,27.0,1.5,17.0,190,96,95,0,88,11.0,20,0,35,2
6,McDonalds,Coffee & Tea,Frappé Chocolate Chip,12 fl oz cup,530,23.0,1.0,14.0,135,76,65,1,67,8.0,15,0,25,4
7,McDonalds,Coffee & Tea,Frappé Chocolate Chip,16 fl oz cup,630,26.0,1.0,17.0,160,91,80,1,81,9.0,15,0,30,4
8,McDonalds,Coffee & Tea,Frappé Chocolate Chip,22 fl oz cup,760,31.0,1.5,20.0,200,111,95,1,99,12.0,20,0,35,6
9,McDonalds,Coffee & Tea,Frappé Mocha,12 fl oz cup,450,18.0,1.0,12.0,125,65,65,1,57,7.0,15,0,20,4


In [11]:
pairings = {
    1: ('Coffee', 'Brewed Coffee'),
    2: ('Nonfat Latte', 'Caffè Latte'),
    3: ('Mocha with Nonfat Milk', 'Caffè Mocha (Without Whipped Cream)'),
    4: ('Frappé Caramel', 'Frappuccino Caramel (Without Whipped Cream)'),
    5: ('Hot Chocolate with Nonfat Milk', 'Hot Chocolate (Without Whipped Cream)'),
    6: ('Regular Iced Coffee', 'Iced Brewed Coffee (With Milk & Classic Syrup)'),
    7: ('Frappé Chocolate Chip', 'Frappuccino Java Chip (Without Whipped Cream)'),
    8: ('Frappé Mocha', 'Frappuccino Mocha (Without Whipped Cream)'),
    9: ('Sweet Tea', 'Shaken Iced Tazo® Tea Lemonade (With Classic Syrup)'),
    10: ('Strawberry Banana Smoothie', 'Strawberry Banana Smoothie'),
    11: ('Nonfat French Vanilla Latte', 'Vanilla Latte (Or Other Flavoured Latte)')
}

In [12]:
def assign_pair(row):
    beverage = row['Beverage'].strip()  # Remove leading/trailing whitespace
    for pair, (mcd_item, sb_item) in pairings.items():
        if (row['Restaurant'] == 'McDonalds' and beverage == mcd_item) or \
           (row['Restaurant'] == 'Starbucks' and beverage == sb_item):
            return pair
    return None

# Apply the function to create a new 'Pair' column
combined_drinks['Pair'] = combined_drinks.apply(assign_pair, axis=1)

combined_drinks[['Beverage', 'Pair', 'Restaurant', 'Size']].sort_values(['Pair', 'Size'])

,Beverage,Pair,Restaurant,Size
0,Coffee,1,McDonalds,12 fl oz cup
31,Brewed Coffee,1,Starbucks,12 fl oz cup
1,Coffee,1,McDonalds,16 fl oz cup
32,Brewed Coffee,1,Starbucks,16 fl oz cup
2,Coffee,1,McDonalds,20 fl oz cup
33,Brewed Coffee,1,Starbucks,20 fl oz cup
21,Nonfat Latte,2,McDonalds,12 fl oz cup
34,Caffè Latte,2,Starbucks,12 fl oz cup
22,Nonfat Latte,2,McDonalds,16 fl oz cup
35,Caffè Latte,2,Starbucks,16 fl oz cup


In [13]:
#ALl mcdonalds frappe's and the smoothie comes in 22oz instead of 20.
def standardize_size(size):
    if isinstance(size, (int, float)):
        if size in [12, 16]:
            return size
        elif size in [20, 22]:
            return 20
    elif isinstance(size, str):
        if '12 fl oz cup' in size or 'Tall' in size:
            return 'Small'
        elif '16 fl oz cup' in size or 'Grande' in size:
            return 'Medium'
        elif '20 fl oz cup' in size or '22 fl oz' in size or 'Venti' in size:
            return 'Large'
    return None

# Apply the function to create a new 'Standardized Size' column
combined_drinks['Standardized Size'] = combined_drinks['Size'].apply(standardize_size)
combined_drinks.to_excel('Combined_paired_drinks.xlsx')

In [14]:
combined_drinks['Drink ID'] = combined_drinks['Pair'].astype(str) + '_' + combined_drinks['Standardized Size'].astype(str)
combined_drinks = combined_drinks.sort_values(['Pair', 'Standardized Size', 'Restaurant'])
combined_drinks.columns

Index(['Restaurant', 'Category', 'Beverage', 'Size', 'Calories',
       'Total Fat (g)', 'Trans Fat (g)', 'Saturated Fat (g)', 'Sodium (mg)',
       'Total Carbohydrates (g)', 'Cholesterol (mg)', 'Dietary Fibre (g)',
       'Sugars (g)', 'Protein (g)', 'Vitamin A (% DV)', 'Vitamin C (% DV)',
       'Calcium (% DV)', 'Iron (% DV)', 'Pair', 'Standardized Size',
       'Drink ID'],
      dtype='object')